In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras import regularizers, optimizers

import numpy as np
from matplotlib import pyplot

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype("float32")
X_test  = X_test.astype("float32")

(X_train, X_valid) = X_train[5000:], X_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000]

In [3]:
print('x_train =', X_train.shape)
print('x_valid =', X_valid.shape)
print('x_test =', X_test.shape)

x_train = (45000, 32, 32, 3)
x_valid = (5000, 32, 32, 3)
x_test = (10000, 32, 32, 3)


In [4]:
mean = np.mean(X_train,axis=(0,1,2,3))
std = np.std(X_train,axis=(0,1,2,3))
X_train = (X_train-mean)/(std+1e-7)
X_valid = (X_valid-mean)/(std+1e-7)
X_test = (X_test-mean)/(std+1e-7)

In [5]:
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_valid = np_utils.to_categorical(y_valid,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [6]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False)
datagen.fit(X_train)

In [7]:
weight_decay = 1e-4
model = Sequential()

# Layer 1 - Conv
model.add(Conv2D(32, (3,3), input_shape=(32,32,3),  padding='same',
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation("relu"))
model.add(BatchNormalization())

# Layer 2 - Conv
model.add(Conv2D(32, (3,3), padding='same',
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation("relu"))
model.add(BatchNormalization())

# MaxPool & Dropout
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

# Layer 3 - Conv
model.add(Conv2D(64, (3,3), padding='same',
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation("relu"))
model.add(BatchNormalization())

# Layer 4 - Conv
model.add(Conv2D(64, (3,3), padding='same',
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation("relu"))
model.add(BatchNormalization())

# MaxPool & Dropout
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

# Layer 5 - Conv
model.add(Conv2D(128, (3,3),  padding='same',
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation("relu"))
model.add(BatchNormalization())

# Layer 6 - Conv
model.add(Conv2D(128, (3,3),  padding='same',
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation("relu"))
model.add(BatchNormalization())

# MaxPool & Dropout
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

# FC 7
model.add(Flatten())
model.add(Dense(10, activation="softmax"))

In [13]:
batch_size = 128                                                          
epochs = 125                                                              
 
checkpointer = ModelCheckpoint(filepath='model.100epochs.hdf5', verbose=1, 
                               save_best_only=True )                      
#optimizer = keras.optimizers.adam(lr=0.0001,decay=1e-6)                   
 
model.compile(loss='categorical_crossentropy',
              optimizer="Adam",
              metrics=['accuracy'])                                                
 
history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), callbacks=[checkpointer], steps_per_epoch=X_train.shape[0] // batch_size, epochs=epochs, verbose=2, validation_data=(X_valid, y_valid))                   

Epoch 1/125

Epoch 00001: val_loss improved from inf to 2.13756, saving model to model.100epochs.hdf5
351/351 - 94s - loss: 1.8135 - accuracy: 0.4180 - val_loss: 2.1376 - val_accuracy: 0.3314
Epoch 2/125

Epoch 00002: val_loss improved from 2.13756 to 1.31174, saving model to model.100epochs.hdf5
351/351 - 96s - loss: 1.3074 - accuracy: 0.5606 - val_loss: 1.3117 - val_accuracy: 0.5994
Epoch 3/125

Epoch 00003: val_loss improved from 1.31174 to 0.97897, saving model to model.100epochs.hdf5
351/351 - 93s - loss: 1.1145 - accuracy: 0.6260 - val_loss: 0.9790 - val_accuracy: 0.6876
Epoch 4/125

Epoch 00004: val_loss improved from 0.97897 to 0.91183, saving model to model.100epochs.hdf5
351/351 - 96s - loss: 0.9765 - accuracy: 0.6738 - val_loss: 0.9118 - val_accuracy: 0.7142
Epoch 5/125

Epoch 00005: val_loss improved from 0.91183 to 0.79410, saving model to model.100epochs.hdf5
351/351 - 93s - loss: 0.8976 - accuracy: 0.7061 - val_loss: 0.7941 - val_accuracy: 0.7550
Epoch 6/125

Epoch 00006